![image](https://i.ibb.co/d03v41b/architecture-diagramxx.png)

In [ ]:
import boto3
import json
import time

region_aws = 'us-east-1'

session = boto3.Session(profile_name='my-profile')
ec2 = session.resource('ec2', region_name=region_aws)
ec2_client = session.client('ec2', region_name=region_aws)
elbv2 = session.client('elbv2', region_name=region_aws)

In [ ]:
# Create VPC
vpc_applic = ec2.create_vpc(CidrBlock='10.10.0.0/16')
vpc_applic.create_tags(Tags=[{"Key": "Name", "Value": "Applicance"}]);

In [ ]:
# Create 2 subnets in VPC Applicance
sn_appliance_transit_a = ec2.create_subnet(
    VpcId=vpc_applic.id,
    CidrBlock='10.10.1.0/24',
    AvailabilityZone=region_aws+'a'
)
sn_appliance_transit_b = ec2.create_subnet(
    VpcId=vpc_applic.id,
    CidrBlock='10.10.2.0/24',
    AvailabilityZone=region_aws+'b'
)

In [ ]:
# Tag subnets
sn_appliance_transit_a.create_tags(Tags=[{"Key": "Name", "Value": "Public-Subnet"}])
sn_appliance_transit_b.create_tags(Tags=[{"Key": "Name", "Value": "Private-Subnet"}])

In [ ]:
# Create route table for the VPC Applicance
rt_appliance_transit_a = vpc_applic.create_route_table()
rt_appliance_transit_b = vpc_applic.create_route_table()

# Tags Route Tables
rt_appliance_transit_a.create_tags(Tags=[{"Key": "Name", "Value": "Public-RT"}])
rt_appliance_transit_b.create_tags(Tags=[{"Key": "Name", "Value": "Private-RT"}])

In [ ]:
# Attach the route table to the subnets
rt_appliance_transit_a.associate_with_subnet(SubnetId=sn_appliance_transit_a.id)
rt_appliance_transit_b.associate_with_subnet(SubnetId=sn_appliance_transit_b.id)

In [ ]:
# Create Internet Gateway
igw = ec2.create_internet_gateway()

# Attach Internet Gateway to VPC Applicance
vpc_applic.attach_internet_gateway(InternetGatewayId=igw.id);

In [ ]:
# Crear una Elastic IP address
elastic_ip_1 = ec2_client.allocate_address(Domain='vpc')

In [ ]:
# Crear un NAT gateway en VPC Applicance
natgateway_1 = ec2_client.create_nat_gateway(
    AllocationId=elastic_ip_1['AllocationId'],
    SubnetId=sn_appliance_transit_a.id
)

In [ ]:
# Añada una ruta a la tabla de enrutamiento de la VPC Applicance para el internet natgateway
rt_appliance_transit_a.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    GatewayId=igw.id
)

In [ ]:
# Añada una ruta a la tabla de enrutamiento de la VPC Applicance para el NAT gateway
rt_appliance_transit_b.create_route(
    DestinationCidrBlock='0.0.0.0/0',
    NatGatewayId=natgateway_1['NatGateway']['NatGatewayId']
)

#### Create EC2 instance

In [27]:
# Create public instance in subnet public
print("En la terminal ejecute el siguiente comando:")
print("    python create_ec2_ssm.py --vpc {} --subnet {} --tag SSM-Public-Instance".format(vpc_applic.id, sn_appliance_transit_a.id))

En la terminal ejecute el siguiente comando:
    python create_ec2_ssm.py --vpc vpc-0a2ca46f02d3021a8 --subnet subnet-09c8a6922c24fae40 --tag SSM-Public-Instance


In [29]:
# Create private instance in subnet private
print("En la terminal ejecute el siguiente comando:")
print("    python create_ec2_ssm.py --vpc {} --subnet {} --tag SSM-Private-Instance".format(vpc_applic.id, sn_appliance_transit_b.id))

En la terminal ejecute el siguiente comando:
    python create_ec2_ssm.py --vpc vpc-0a2ca46f02d3021a8 --subnet subnet-039213c0c26355992 --tag SSM-Private-Instance


In [35]:
# List all instances
for instance in ec2.instances.all():
    print(instance.id, instance.state, instance.tags)

i-0ff007cb78ed339c2 {'Code': 16, 'Name': 'running'} [{'Key': 'SSM', 'Value': 'true'}, {'Key': 'Name', 'Value': 'SSM-Private-Instance'}]
i-0c7b0f824facf448b {'Code': 16, 'Name': 'running'} [{'Key': 'Name', 'Value': 'SSM-Public-Instance'}, {'Key': 'SSM', 'Value': 'true'}]
i-078e399b6bc4f1121 {'Code': 16, 'Name': 'running'} [{'Key': 'Name', 'Value': 'SSM-Public-Instance-2'}, {'Key': 'SSM', 'Value': 'true'}]


In [36]:
print("Para conectarse a la instancia ejecute: \n    aws ssm start-session --target i-0e0e0e0e0e0e0e0e0")

Para conectarse a la instancia ejecute: 
    aws ssm start-session --target i-0e0e0e0e0e0e0e0e0
